# 1. Import modules and functions

In [5]:
os.getcwd()

'/home/alex/projects/ocr'

In [ ]:
import os
import sys
from pathlib import Path
import glob
import pandas as pd
from datasets import Dataset, load_from_disk, concatenate_datasets, DatasetInfo, DatasetDict
from PIL import Image
#os.chdir("..")
#os.chdir("ocr")
from src.extract_create import *

import importlib
import src.extract_create

# Reload the module to ensure changes are picked up
importlib.reload(src)
input_folder = Path("data/raw/")
output_folder = Path("data/processed")

# 2. OLD: Extract text regions and crop images

In [ ]:

extract_and_create(input_folder, output_folder)

# OLD: Create the combined dataset and upload it to HuggingFace

In [15]:
text_dir = "data/processed/texts"
image_dir = "data/processed/images"

dataframes = [pd.read_csv(x) for x in glob.glob(f"{text_dir}/*.csv")]


In [ ]:
dataframes
datasets = [process_dataframe(df,image_dir) for df in dataframes]
combined_dataset = concatenate_datasets(datasets)

# Save the combined dataset
combined_dataset.save_to_disk("data/processed/dataset_p40")

# NEW: Images already in correct format, now need to extract texts from csv table

In [4]:
# load csv
p0_40_singlecol_df = pd.read_csv("data/raw_singlecol/meeting-minutes-40pages-gt.csv")
p40_80_singlecol_df = pd.read_csv("data/raw_singlecol/48-87_prediction.csv")

p0_80_singlecol_df = pd.concat([p0_40_singlecol_df, p40_80_singlecol_df], ignore_index=True)


text_dir = "data/processed_singlecol/texts"
image_dir = "data/processed_singlecol/images"

dataframes = p0_80_singlecol_df[["id", "transcription"]].copy()
dataframes

,id,transcription
0,2-0,陳甲
1,2-1,楊甲
2,2-2,王興廣叫
3,2-3,陳江娘呌
4,2-4,貳文貳鈸願過汝夫數内以還興廣之錢否答曰願
...,...,...
2938,087-28,陈金诉谓前期的唤盛祖到堂证实秦寻他不遇乞再限
2939,087-29,陳全观
2940,087-30,連敦观
2941,087-31,李五芳和息前视李互法


In [9]:
# # Function to map the image loading for each row
def process_images(example):
    if("singlecol" in image_dir):
        image_filename = example['id'] + '.jpg' 
    else:
        image_filename = example['identifier'] + '.png' 
    image_path = os.path.join(image_dir, image_filename)  # 'identifier' is the image file name
    example['image'] = Image.open(image_path)
    return example


In [11]:

# Apply the image loading function
dataset = Dataset.from_pandas(dataframes)
dataset = dataset.map(process_images)
# Save the combined dataset
#dataset.save_to_disk("data/processed_singlecol/dataset_p40")
dataset.save_to_disk("data/processed_singlecol/dataset_p80")
combined_dataset = dataset

Saving the dataset (1/1 shards): 100%|██████████| 2943/2943 [00:00<00:00, 32036.95 examples/s]


# Deprecated: Trying to create an only single column dataset using the information given in the xml files -- coordinates wrong

In [44]:
# input_folder = Path("data/raw/")
# output_folder = Path("data/processed_singlecol")
# extract_and_create_singlecol(input_folder, output_folder)

In [47]:
# text_dir = "data/processed_singlecol/texts"
# image_dir = "data/processed_singlecol/images"
# dataframes = [pd.read_csv(x) for x in glob.glob(f"{text_dir}/*.csv")]
# # Combine datasets from each dataframe
# datasets = [process_dataframe(df,image_dir) for df in dataframes]
# combined_dataset = concatenate_datasets(datasets)
# combined_dataset.save_to_disk("data/processed_singlecol/data_p40_singlecol")

## Spliting into train and validation set

In [37]:
train_test_split = combined_dataset.train_test_split(test_size=0.2)

split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

In [ ]:
### Saving to HF hub

In [55]:
with open("data/tokens/token.txt") as f:
    token = f.read()

In [56]:
split_dataset.push_to_hub("culturalheritagenus/gongguan_ocr_p40_singlecol", token=token)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/culturalheritagenus/gongguan_ocr_p40_singlecol/commit/311d7ad25ea95473b807d8953faa6bfc60adfeec', commit_message='Upload dataset', commit_description='', oid='311d7ad25ea95473b807d8953faa6bfc60adfeec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/culturalheritagenus/gongguan_ocr_p40_singlecol', endpoint='https://huggingface.co', repo_type='dataset', repo_id='culturalheritagenus/gongguan_ocr_p40_singlecol'), pr_revision=None, pr_num=None)